In [31]:
# update signals.json for web screener
# Daily Looping
from backtesting import Backtest, Strategy
import yfinance as yf
import json
import pandas as pd
import datetime
import pandas_ta as ta
import tqdm
import subprocess
route = "../Database/"
filemap = {
    "NYSE"  : route + "../Database/zackRanks_NYSE.json", # 1352
    "NASDAQ": route + '../Database/zackRanks_NASDAQ.json', # small:1255; mid:330; large:72
    "SP500" : route + "../Database/zackRanks_SP500.json", # small: 1352
   "Yuanta": route + "zackRanks_yuanta.json",
   "buy": route + "buy.json",
   "sell": route + "sell.json",
   "portfolio": route + "zackRanks_portfolio.json"
}
def get_tickers(file="SP500"):
    dict = pd.read_csv(filemap[file], header=None)
    dict = dict.iloc[:, 0]
    return dict

def WPRbt(data):
    period = 12
    max = data.Close.rolling(period).max()
    min = data.Close.rolling(period).min()
    wpr = (max - data.Close)/(max - min) * (-100) 
    return wpr

def check_WPR_2MA_without_DB(ticker, test=False, period_long_ma=28, period_short_ma=14, upb=-20, lob=-80):
    a = yf.download(ticker, start=datetime.datetime.now() - datetime.timedelta(days=50), interval="1d", progress=False)

    # a["WPR"] = WPRbt(a)
    a["MAX"] = a.High.rolling(12).max()
    a["MIN"] = a.Low.rolling(12).min()
    a["WPRs"] = (a.MAX - a.Close)/(a.MAX - a.MIN) * (-100)
    a["long_MA"] = a.Close.ewm(span=period_long_ma, adjust=False, min_periods=period_long_ma).mean()
    a['short_MA'] = a.Close.ewm(span=period_short_ma, adjust=False, min_periods=period_short_ma).mean()
    long0 = a['long_MA'][-1]
    long1 = a['long_MA'][-2]
    long2 = a['long_MA'][-3]
    long3 = a['long_MA'][-4]
    # =======================
    short0 = a['short_MA'][-1]
    short1 = a['short_MA'][-2]
    short2 = a['short_MA'][-3]
    short3 = a['short_MA'][-4]
    # =======================
    wpr0 = a['WPRs'][-1]
    wpr1 = a['WPRs'][-2]
    wpr2 = a['WPRs'][-3]
    wpr3 = a['WPRs'][-4]
    check_ma_percent = 0.0000# for PAYX at 2024-02-08, 0.005 return 4(meaning doesn't work ) while 0.006 works
    bear_market = (long0 - short0 > long0*check_ma_percent) and (long1 - short1 > long1*check_ma_percent) and (long2 - short2 > long2*check_ma_percent)
    bull_market = (long0 - short0 < long0*-check_ma_percent) and (long1 - short1 < long1*-check_ma_percent) and (long2 - short2 < long2*-check_ma_percent)
    if test:
        print(a[['Close', "short_MA", "long_MA", "WPRs", "MAX"]])
        print(wpr0, wpr1, wpr2, wpr3)
        print("quick: ", [short3, short2, short1, short0])
        print("slow: ", [long3, long2, long1, long0])
    # current signal
    if((wpr0 < upb) and (wpr1 > upb) and bear_market):
        # print(ticker, "cross above"):
            # sell signal
        return 1
    elif ((wpr0 > lob) and (wpr1 < lob)) and bull_market:
        # buy signal
        return 2
    # check for yesterday or the day before yesterday buy/sell signal
    elif((wpr1 < upb) and (wpr2 > upb) and bear_market):
        return 1.5
    elif((wpr1 > lob) and (wpr2 < lob) and bull_market):
        return 2.5
    elif((wpr2 > lob) and (wpr3 < lob) and bull_market) or ((wpr2 > upb) and (wpr3 < upb) and bear_market):
        return 3

    else:
        return 0

def mains(loops = ["NYSE", "NASDAQ", "SP500"], updateDatabase = False):
    all_signals = {}
    for k in loops:
        print("*"*20, k, "*"*20)

        with open(filemap[k], 'r') as f:
            python_dict = json.load(f)
        a = list(python_dict.keys())
        if k in ['portfolio', "SP500", "Yuanta"]:
            tickers = a[:]
        elif k == "NYSE":
            tickers = a[1:1352]
        elif k == "NASDAQ":
            tickers = a[1:1255]
        with tqdm.tqdm(total=len(tickers)) as pbar:
            res = {}
            for ticker in tickers:
                res[ticker] = check_WPR_2MA_without_DB(ticker)
                pbar.update(1)  # Update the progress bar

        # s = {{i:res[i]} for i in res.keys() if res[i] in [1, 2]}
        for key, value in res.items():
            if value in [1, 2]:
                all_signals[key] = value
                    
        if sum(list(res.values())) == 0:
            print("there's nothing with signal")
    print(all_signals)

    time = (datetime.datetime.now() - datetime.timedelta(days=1)).strftime("%Y-%m-%d")

    with open("../Database/signals.json", "r") as f:
        python_dict_signal = json.load(f)
    python_dict_signal[time] = all_signals
    with open("../Database/signals.json", "w") as f:
        json.dump(python_dict_signal, f , indent=4)

updateDatabase = True

loops = ["Yuanta", "SP500", "NASDAQ", "NYSE"]
mains(updateDatabase=updateDatabase, loops=loops)


******************** Yuanta ********************


100%|██████████| 18/18 [00:02<00:00,  7.39it/s]


{'TSLA': 1}
******************** SP500 ********************


100%|██████████| 499/499 [01:09<00:00,  7.20it/s]


{'TSLA': 1, 'MSFT': 2, 'MA': 2, 'HON': 1, 'CDNS': 2, 'USB': 1, 'PSX': 2, 'AZO': 2, 'PSA': 1, 'DLR': 2, 'SYY': 2, 'GIS': 1, 'KMI': 1, 'DVN': 1, 'PPL': 1, 'CLX': 2, 'TXT': 2, 'JBHT': 2, 'MRO': 1, 'TAP': 1}
******************** NASDAQ ********************


100%|██████████| 1254/1254 [02:23<00:00,  8.73it/s]


{'TSLA': 1, 'MSFT': 2, 'MA': 2, 'HON': 1, 'CDNS': 2, 'USB': 1, 'PSX': 2, 'AZO': 2, 'PSA': 1, 'DLR': 2, 'SYY': 2, 'GIS': 1, 'KMI': 1, 'DVN': 1, 'PPL': 1, 'CLX': 2, 'TXT': 2, 'JBHT': 2, 'MRO': 1, 'TAP': 1, 'BLUE': 1, 'WKHS': 1, 'APA': 1, 'TSP': 1, 'EVGO': 1, 'BYND': 1, 'EU': 2, 'NEOG': 1, 'ADMA': 2, 'ABCL': 1, 'CHKP': 2, 'ACDC': 1, 'ZNTL': 1, 'RLAY': 1, 'FIVE': 1, 'TRIN': 1, 'ASLE': 1, 'CLDX': 1, 'PNFP': 1, 'ABOS': 1, 'SNBR': 1, 'IMUX': 1, 'LOGI': 1, 'CRMD': 1, 'PXMD': 1, 'EXPO': 1, 'AMTX': 1, 'NCMI': 1}
******************** NYSE ********************


100%|██████████| 1351/1351 [03:17<00:00,  6.83it/s]

{'TSLA': 1, 'MSFT': 2, 'MA': 2, 'HON': 1, 'CDNS': 2, 'USB': 1, 'PSX': 2, 'AZO': 2, 'PSA': 1, 'DLR': 2, 'SYY': 2, 'GIS': 1, 'KMI': 1, 'DVN': 1, 'PPL': 1, 'CLX': 2, 'TXT': 2, 'JBHT': 2, 'MRO': 1, 'TAP': 1, 'BLUE': 1, 'WKHS': 1, 'APA': 1, 'TSP': 1, 'EVGO': 1, 'BYND': 1, 'EU': 2, 'NEOG': 1, 'ADMA': 2, 'ABCL': 1, 'CHKP': 2, 'ACDC': 1, 'ZNTL': 1, 'RLAY': 1, 'FIVE': 1, 'TRIN': 1, 'ASLE': 1, 'CLDX': 1, 'PNFP': 1, 'ABOS': 1, 'SNBR': 1, 'IMUX': 1, 'LOGI': 1, 'CRMD': 1, 'PXMD': 1, 'EXPO': 1, 'AMTX': 1, 'NCMI': 1, 'NOK': 2, 'BMY': 1, 'AMC': 1, 'LYG': 1, 'ABR': 1, 'HSY': 2, 'ACI': 1, 'UDR': 1, 'UPS': 1, 'ES': 1, 'VSH': 1, 'ENLC': 1, 'WTI': 1, 'LLAP': 1, 'DTE': 1, 'PBF': 2, 'CIG': 1, 'SXC': 2, 'OR': 2, 'LEN': 2, 'MKC': 1, 'RY': 1, 'PRGO': 1, 'DBI': 2, 'ASB': 1, 'NRGV': 1, 'NJR': 1, 'SNV': 1, 'RCUS': 1, 'SUI': 1, 'OUST': 1, 'BSBR': 1, 'BAK': 1, 'PGTI': 1, 'SOL': 1, 'SLGN': 2, 'HRTG': 1, 'DRQ': 1, 'AVNS': 1}


In [19]:
# Update json files for zackRanks raw files
# Daily Looping
from backtesting import Backtest, Strategy
import yfinance as yf
import json
import pandas as pd
import datetime
import pandas_ta as ta
import tqdm
import subprocess
route = "../Database/"
filemap = {
    "NYSE"  : route + "../Database/zackRanks_NYSE.json", # 1352
    "NASDAQ": route + '../Database/zackRanks_NASDAQ.json', # small:1255; mid:330; large:72
    "SP500" : route + "../Database/zackRanks_SP500.json", # small: 1352
   "Yuanta": route + "zackRanks_yuanta.json",
   "buy": route + "buy.json",
   "sell": route + "sell.json",
   "portfolio": route + "zackRanks_portfolio.json"
}

def mains(today=0, loops = ["NYSE", "Yuanta", "NASDAQ", "SP500"]):
    for k in loops:
        print("*"*20, k, "*"*20)
        with open(filemap[k], 'r') as f:
            python_dict = json.load(f)
            a = list(python_dict.keys())
            with tqdm.tqdm(total=len(a)) as pbar:
                for ticker in a:
                    command = "zacks-api " + ticker
                    result = subprocess.check_output(command, shell=True)
                    dict = eval(result.decode(("utf-8")))
                    tick = dict["ticker"]
                    rank = dict["zacksRank"]
                    if today == 0:
                        dat = dict["updatedAt"].split("T")[0]
                    else:
                        dat = datetime.datetime.now() - datetime.timedelta(days=today)
                    python_dict[tick][dat] = rank
                    pbar.update(1)
        with open(filemap[k], 'w') as f:
            json.dump(python_dict, f, indent=4)


loops = ["NYSE", "Yuanta", "NASDAQ", "SP500", "portfolio"]
# SP500: 
# today: default_day = 0
mains(today=0, loops=loops)

******************** SP500 ********************


100%|██████████| 499/499 [10:13<00:00,  1.23s/it]


In [ ]:
# update Tickers/{}.csv
import datetime
import pandas as pd
import yfinance as yf
route = "../Database/"
filemap = {
    "NYSE"  : route + "../Database/zackRanks_NYSE.json", # 1352
    "NASDAQ": route + '../Database/zackRanks_NASDAQ.json', # small:1255; mid:330; large:72
    "SP500" : route + "../Database/zackRanks_SP500.json", # small: 1352
   "Yuanta": route + "zackRanks_yuanta.json",
   "buy": route + "buy.json",
   "sell": route + "sell.json",
   "portfolio": route + "zackRanks_portfolio.json"
}
def daily_add_datetime_minus_1(loops=list(filemap.keys())):
    for kys in loops:
        with open(filemap[kys], "r") as f:
            python_dict = json.load(f)
            a = list(python_dict.keys())
        i = 0
        for ticker in a:
            df = pd.read_csv(f"../Database/Tickers/{ticker}.csv", index_col="Date")
            start = datetime.datetime.now()-datetime.timedelta(days=1)
            ftime = start.strftime("%Y-%m-%d")
            try:
                b = df.loc[ftime]
            except KeyError:
                a = yf.download("A", start=start)
                a.index = [ftime]
                df = pd.concat([df, a])
                df.index.name = "Date"
                df.to_csv("../Database/Tickers/A.csv", index="Date")
                if i == 0:
                    print(f"added on {kys}")
                    i += 1
daily_add_datetime_minus_1()                

In [ ]:
# initialization of Tickers/{}.csv
import datetime
import pandas as pd
import yfinance as yf
route = "../Database/"
filemap = {
    "NYSE"  : route + "../Database/zackRanks_NYSE.json", # 1352
    "NASDAQ": route + '../Database/zackRanks_NASDAQ.json', # small:1255; mid:330; large:72
    "SP500" : route + "../Database/zackRanks_SP500.json", # small: 1352
   "Yuanta": route + "zackRanks_yuanta.json",
   "buy": route + "buy.json",
   "sell": route + "sell.json",
   "portfolio": route + "zackRanks_portfolio.json"
}
def database_initialization():
    for kys in ["NYSE", "NASDAQ", "SP500"]:
        with open(filemap[kys], "r") as f:
            python_dict = json.load(f)
            a = list(python_dict.keys())
        a = list(python_dict.keys())
        for ticker in a:
            print(ticker)
            df = yf.download(ticker, progress=False)
            df.to_csv(f"../Database/Tickers/{ticker}.csv")
        
database_initialization()                
